Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

__Also, please write how much time it took you to finish the homework.__ This will not affect your grade in any way and is used for statistical purposes.

In [1]:
TIME_SPENT = "00h00m"

---

## Homework 3
### Word Embeddings

Welcome to Homework 3! 

The homework contains several tasks. You can find the amount of points that you get for the correct solution in the task header. Maximum amount of points for each homework is _four_.

The **grading** for each task is the following:
- correct answer - **full points**
- insufficient solution or solution resulting in the incorrect output - **half points**
- no answer or completely wrong solution - **no points**

Even if you don't know how to solve the task, we encourage you to write down your thoughts and progress and try to address the issues that stop you from completing the task.

When working on the written tasks, try to make your answers short and accurate. Most of the times, it is possible to answer the question in 1-3 sentences.

When writing code, make it readable. Choose appropriate names for your variables (`a = 'cat'` - not good, `word = 'cat'` - good). Avoid constructing lines of code longer than 100 characters (79 characters is ideal). If needed, provide the commentaries for your code, however, a good code should be easily readable without them :)

Finally, all your answers should be written only by yourself. If you copy them from other sources it will be considered as an academic fraud. You can discuss the tasks with your classmates but each solution must be individual.

<font color='red'>**Important!:**</font> **before sending your solution, do the `Kernel -> Restart & Run All` to ensure that all your code works.**

In [2]:
from pprint import pprint

import fasttext
from datasets import load_dataset
from pathlib import Path
from sklearn.metrics import classification_report

from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

You are going to work with the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) again and use it to train a [Fasttext classifier](https://fasttext.cc/docs/en/supervised-tutorial.html). To spare your time loading the dataset, we will do it using [HuggingFace datasets library](https://huggingface.co/docs/datasets/index.html). We will also split train set into 20,000 samples for training and 5,000 samples for validation.

Working with `datasets` is easy. After loading and splitting the dataset, each split can be accessed by its index in the same order as defined while loading. For example, training set will be `imdb_dataset[0]`, validation set `imdb_dataset[1]`, test set `imdb_dataset[2]`.

Each individual sample can be also accessed with the corresponding index. For example, to take the first sample from the training set, we should write `imdb_dataset[0][0]`.

Each sample is a dictionary, that has `'text'` and `'label'`.

You can play around with the dataset to better see how it works.

In [3]:
imdb_dataset = load_dataset('imdb', split=['train[:10000]+train[15000:]', 'train[10000:15000]', 'test'])

Reusing dataset imdb (/home/utlab/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
imdb_dataset

[Dataset({
     features: ['text', 'label'],
     num_rows: 20000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 5000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 })]

## Task 1. Preprocess the text (0.5 points)

To train a Fasttext classifier, the dataset should be written into text files, where each line corresponds to one example. Each line must also start with `__label__` prefix followed by the label, e.g. label `0` would be `__label__0` for Fasttext. This technical part was implemented for you. You can read more about it [here](https://fasttext.cc/docs/en/supervised-tutorial.html#getting-and-preparing-the-data).

You task is to preprocess the text. You can use any preprocessing tool that we've learned so far. The minimum preprocessing should do:
- tokenize
- lowercase
- remove stopwords
- remove punctuation

The preprocessed text should be represented by a string where each token is separated by the whitespace. For example: `someone actually gave movie stars high chance need immediate professional help`. 

In [5]:
def convert_to_fasttext(dataset, file_path):
    english_stop = stopwords.words('english')  # set of English stopwords
    file_path = Path(file_path)
    with open(file_path, 'w', encoding='utf-8') as f:  # open file for writing
        for item in dataset:
            label = item['label']  # label either 0 or 1
            text = item['text']  # movie review itself

            # transform text variable to contain preprocessed text
            # the label will be attached later

            ### YOUR CODE HERE
            from string import punctuation
            text = simple_preprocess(text)
            text = [word for word in text if word not in punctuation]
            text = [word for word in text if word not in english_stop]

            f.write(f"__label__{label} {text}\n")  # write into the file


In [6]:
# Write train, validation and test sets
convert_to_fasttext(imdb_dataset[0], 'imdb.train')
convert_to_fasttext(imdb_dataset[1], 'imdb.valid')
convert_to_fasttext(imdb_dataset[2], 'imdb.test')

In [7]:
# Read the validation and test files for prediction
valid_texts = [' '.join(line.split(' ')[1:]) for line in Path('imdb.valid').read_text().split('\n')][:-1]
test_texts = [' '.join(line.split(' ')[1:]) for line in Path('imdb.test').read_text().split('\n')][:-1]

# Validation and test labels for classification report
valid_trues = ['__label__' + str(item['label']) for item in imdb_dataset[1]]
test_trues = ['__label__' + str(item['label']) for item in imdb_dataset[2]]

## Task 2.1 Train the classifier (1.5 points)

Read through the [Fasttext classification tutorial](https://fasttext.cc/docs/en/supervised-tutorial.html#our-first-classifier). Try training different models with different parameters to achieve the highest score on the validation set. Then, see the results of the best model on the test set.

In [8]:
# Put the parameters here
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

lrs = [0.3, 0.5, 0.9, 1.0]
epochs = [10, 20, 30]
wordNgrams = [1, 2, 3]


def print_results(N, p, r):
    print("# of Sample:", N)
    print("Accuracy:", p)
    print("Recall:", r)


best_model = ''
best_acc = 0
tmp_acc = 0
for lr in lrs:
    for epoch in epochs:
        for ngram in wordNgrams:
            print(f'Parameter: [lr={lr}, epoch={epoch}, wordNgrams={ngram}]')
            model = fasttext.train_supervised(
                input='imdb.train',
                lr=lr,
                epoch=epoch,
                wordNgrams=ngram)
            result = model.test('imdb.valid')
            tmp_acc = result[1]
            if tmp_acc > best_acc:
                best_acc = tmp_acc
                best_model = f'[lr={lr}, epoch={epoch}, wordNgrams={ngram}]'

print(f'Best Accuracy: {best_acc} \n Best model:{best_model}')
print_results(*result)


Parameter: [lr=0.3, epoch=10, wordNgrams=1]


Read 2M words
Number of words:  74962
Number of labels: 2
Progress: 100.0% words/sec/thread: 1084055 lr:  0.000000 avg.loss:  0.139920 ETA:   0h 0m 0s


Parameter: [lr=0.3, epoch=10, wordNgrams=2]


Read 2M words
Number of words:  74962
Number of labels: 2
Progress:  99.4% words/sec/thread:  500329 lr:  0.006178 avg.loss:  0.098238 ETA:   0h 0m 0s 13.8% words/sec/thread:  547158 lr:  0.258696 avg.loss:  0.435947 ETA:   0h 0m 5s 53.5% words/sec/thread:  504017 lr:  0.139382 avg.loss:  0.168963 ETA:   0h 0m 3s 73.1% words/sec/thread:  502599 lr:  0.080812 avg.loss:  0.128290 ETA:   0h 0m 1s 88.5% words/sec/thread:  502123 lr:  0.034606 avg.loss:  0.107710 ETA:   0h 0m 0s words/sec/thread:  500520 lr:  0.001870 avg.loss:  0.096942 ETA:   0h 0m 0s

Parameter: [lr=0.3, epoch=10, wordNgrams=3]


...

Read 2M words
Number of words:  74962
Number of labels: 2
Progress: 100.0% words/sec/thread:  364454 lr:  0.000000 avg.loss:  0.024734 ETA:   0h 0m 0s 275328 lr:  0.992270 avg.loss:  0.453194 ETA:   0h 0m38s  9.1% words/sec/thread:  250850 lr:  0.908505 avg.loss:  0.236472 ETA:   0h 0m38s 30.0% words/sec/thread:  317722 lr:  0.700229 avg.loss:  0.078752 ETA:   0h 0m23s 42.0% words/sec/thread:  328372 lr:  0.579853 avg.loss:  0.057025 ETA:   0h 0m18s 53.7% words/sec/thread:  342100 lr:  0.463450 avg.loss:  0.044768 ETA:   0h 0m14s 67.7% words/sec/thread:  351956 lr:  0.323190 avg.loss:  0.035691 ETA:   0h 0m 9s 78.1% words/sec/thread:  360567 lr:  0.219221 avg.loss:  0.031247 ETA:   0h 0m 6s 95.9% words/sec/thread:  365755 lr:  0.040859 avg.loss:  0.025738 ETA:   0h 0m 1s


Parameter: [lr=1.0, epoch=30, wordNgrams=3]


Read 2M words
Number of words:  74962
Number of labels: 2


Best Accuracy: 0.8764 
 Best model:[lr=0.3, epoch=20, wordNgrams=3]
# of Sample: 5000
Accuracy: 0.874
Recall: 0.874


Progress: 100.0% words/sec/thread:  237852 lr:  0.000000 avg.loss:  0.021480 ETA:   0h 0m 0s


In [9]:
# Precision at one and recall at one scores
model.test('imdb.valid')

(5000, 0.874, 0.874)

In [10]:
# Sklearn classification report for validation
valid_preds = model.predict(valid_texts)[0]
print(classification_report(valid_trues, valid_preds, digits=4))

              precision    recall  f1-score   support

  __label__0     0.8740    0.8740    0.8740      2500
  __label__1     0.8740    0.8740    0.8740      2500

    accuracy                         0.8740      5000
   macro avg     0.8740    0.8740    0.8740      5000
weighted avg     0.8740    0.8740    0.8740      5000



In [11]:
# Sklearn classification report for test
test_preds = model.predict(test_texts)[0]
print(classification_report(test_trues, test_preds, digits=4))

              precision    recall  f1-score   support

  __label__0     0.8792    0.8757    0.8774     12500
  __label__1     0.8762    0.8797    0.8779     12500

    accuracy                         0.8777     25000
   macro avg     0.8777    0.8777    0.8777     25000
weighted avg     0.8777    0.8777    0.8777     25000



## Task 2.2 Describe the results (0.5 point)

- Briefly describe which parameters you tried and which parameters you ended up using for the final model.
- Which parameter had the most impact on the model's performance?
- What are the scores for your best model?

**(A) :** For optmizing fastText model, I try to fineturn `learning rate` `[0.3, 0.5, 0.9, 1.0]`,`epoch` `[10, 20, 30]`, and `number of ngrams` `[1, 2, 3]`.In the end, the model with `[lr=0.3, epoch=20, wordNgrams=3]` has best performence. In the other hands, `Learning Rate` had most impact on the model. In genernal, `0.874` is the best score in my model.

## Task 3.1 Automatic hyperparameter optimization (0.5 points)

Read about [Automatic hyperparameter optimization](https://fasttext.cc/docs/en/autotune.html) here.

Autotune your model on the validation set (this will take around five minutes to finish).

Look at the results on the validation and test sets.

In [13]:
# Put the parameters here
auto_model = fasttext.train_supervised(input='imdb.train', lr=0.3, epoch=20, wordNgrams=3)

Read 2M words
Number of words:  74962
Number of labels: 2
Progress: 100.0% words/sec/thread:  244408 lr:  0.000000 avg.loss:  0.057624 ETA:   0h 0m 0s 55.6% words/sec/thread:  236237 lr:  0.133061 avg.loss:  0.099612 ETA:   0h 0m13s 0.082079 ETA:   0h 0m 9s 76.3% words/sec/thread:  240828 lr:  0.071183 avg.loss:  0.074168 ETA:   0h 0m 7s


In [14]:
# Sklearn classification report for validation
auto_valid_preds = auto_model.predict(valid_texts)[0]
print(classification_report(valid_trues, auto_valid_preds, digits=4))

              precision    recall  f1-score   support

  __label__0     0.8784    0.8728    0.8756      2500
  __label__1     0.8736    0.8792    0.8764      2500

    accuracy                         0.8760      5000
   macro avg     0.8760    0.8760    0.8760      5000
weighted avg     0.8760    0.8760    0.8760      5000



In [15]:
# Sklearn classification report for test
auto_test_preds = auto_model.predict(test_texts)[0]
print(classification_report(test_trues, auto_test_preds, digits=4))

              precision    recall  f1-score   support

  __label__0     0.8822    0.8762    0.8791     12500
  __label__1     0.8770    0.8830    0.8800     12500

    accuracy                         0.8796     25000
   macro avg     0.8796    0.8796    0.8796     25000
weighted avg     0.8796    0.8796    0.8796     25000



## Task 3.2 Look at the model's parameters (0.5 points)

Check the parameters of the autotuned model.

- Briefly decribe how different they are from your previous model
- What was the most surprising change?
- Did autotuned model perform better then your manually tuned model?

In [16]:
print('Learning rate:', auto_model.lr)
print('Word vector dimention:', auto_model.dim)
print('Number of epochs:', auto_model.epoch)
print('Word n-grams:', auto_model.wordNgrams)
print('Minimum length of char n-gram:', auto_model.minn)
print('Maximum length of char n-gram:', auto_model.maxn)
print('Size of context window:', auto_model.ws)

Learning rate: 0.3
Word vector dimention: 100
Number of epochs: 20
Word n-grams: 3
Minimum length of char n-gram: 0
Maximum length of char n-gram: 0
Size of context window: 5


**(A) :** The result of model is not that different to mine, however it spend less time to mine.

## Task 4. Play with word vectors (0.5 points)

Since the Fasttext classifier trained the word embeddings from scratch based on our dataset, you can explore them as any other word embeddings.

Read [this tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html#nearest-neighbor-queries) on word representations, and try to explore a little the word vectors, for example try to see nearest neighbors of some words or get some word analogies. You can also try different movie related words to see how they are compared to more general words.

Probably, the quality of these word embeddings will not be very good since our dataset is relatively small, but it is still good idea to check yourself what did the model learn in the end.

In [19]:
### YOUR CODE HERE
pprint(model.get_nearest_neighbors('dog'))

[(0.0, "'one',"),
 (0.0, "'movie',"),
 (0.0, '</s>'),
 (0.0, "'even',"),
 (0.0, "'like',"),
 (0.0, "'would',"),
 (0.0, "'time',"),
 (0.0, "'burg',"),
 (0.0, "'herts',"),
 (0.0, "['tough',")]


In [21]:
pprint(model.get_analogies('pet', 'dog', 'humen'))

[(0.0, "'one',"),
 (0.0, "'movie',"),
 (0.0, '</s>'),
 (0.0, "'even',"),
 (0.0, "'like',"),
 (0.0, "'would',"),
 (0.0, "'time',"),
 (0.0, "'burg',"),
 (0.0, "'herts',"),
 (0.0, "['tough',")]
